In [11]:
import torch
torch.backends.cudnn.benchmark = True

import cv2
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from apis.inference import inf_single_pose, process_yolo, vis_pose_result

from mmcv import Config
from mmcv.cnn import fuse_conv_bn
from mmcv.parallel import MMDataParallel, collate
from mmcv.runner import load_checkpoint
from mmpose.models import build_posenet

In [4]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /home/nami-labs/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv         

In [5]:
cfg_file = 'configs/pose/top_down/darkpose/coco/hrnet_w32_coco_256x192_dark.py'
ckpt_file = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192_dark-07f147eb_20200812.pth'

In [15]:
cfg = Config.fromfile(cfg_file)
pose_model = build_posenet(cfg.model)
    
load_checkpoint(pose_model, ckpt_file, map_location='cpu')

pose_model = fuse_conv_bn(pose_model)
model_parallel = MMDataParallel(pose_model, device_ids=[0])

Use load_from_http loader
2021-04-06 21:25:31,030 - mmpose - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.0.0.0.conv1.weight, head.0.0.0.bn1.weight, head.0.0.0.bn1.bias, head.0.0.0.bn1.running_mean, head.0.0.0.bn1.running_var, head.0.0.0.bn1.num_batches_tracked, head.0.0.0.conv2.weight, head.0.0.0.bn2.weight, head.0.0.0.bn2.bias, head.0.0.0.bn2.running_mean, head.0.0.0.bn2.running_var, head.0.0.0.bn2.num_batches_tracked, head.0.0.0.conv3.weight, head.0.0.0.bn3.weight, head.0.0.0.bn3.bias, head.0.0.0.bn3.running_mean, head.0.0.0.bn3.running_var, head.0.0.0.bn3.num_batches_tracked, head.0.0.0.downsample.0.weight, head.0.0.0.downsample.1.weight, head.0.0.0.downsample.1.bias, head.0.0.0.downsample.1.running_mean, head.0.0.0.downsample.1.running_var, head.0.0.0.downsample.1.num_batches_tracked, head.0.1.0.conv1.weight, head.0.1.0.bn1.weight, head.0.1.0.bn1.bias, head.0.1.0.bn1.running_mean, head.0.1.0.bn1.running_var, head.0.1.0.b

In [7]:
img = cv2.imread('images/run.jpg', cv2.IMREAD_COLOR)
cv2.imshow('image', img)

cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [13]:
det_results = yolo_model(img)
det_results_xyxy, det_results_xywh = process_yolo(det_results, score_thr=0.3)

pose_results = inf_single_pose(pose_model, cfg, img, det_results_xywh, det_results_xyxy)
vis_result = vis_pose_result(pose_model, img, pose_results)

cv2.imshow('image', vis_result)

cv2.waitKey(0) 
cv2.destroyAllWindows() 